In [1]:
import os
os.environ['KERAS_BACKEND']='tensorflow'
from PIL import Image
import numpy as np

In [2]:
def load_data():
    data = np.empty((42000,1,28,28),dtype="float32")
    label = np.empty((42000,),dtype="uint8")

    imgs = os.listdir("/home/keras/Desktop/keras/mnist")
    num = len(imgs)  # 42000
    for i in range(num):
        img = Image.open("/home/keras/Desktop/keras/mnist/"+imgs[i])
        arr = np.asarray(img,dtype="float32")
        data[i,:,:,:] = arr
        label[i] = int(imgs[i].split('.')[0])
    return data,label

In [3]:
from __future__ import absolute_import
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras.utils import np_utils, generic_utils
from six.moves import range
import random
np.random.seed(1024) 

Using TensorFlow backend.


In [4]:
X_data, Y_data  = load_data()
# 打亂資料
index = [i for i in range(len(X_data))]
random.shuffle(index)
X_data = X_data[index]
Y_data = Y_data[index]
print(X_data.shape[0], ' samples')

42000  samples


In [5]:
X_train, X_test = X_data[:30000,:,:,:], X_data[30000:,:,:,:]
Y_train, Y_test = Y_data[:30000,], Y_data[30000:,]


In [6]:
# input image dimensions
img_rows = X_train.shape[2]
img_cols = X_train.shape[3]
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1 )
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1 )
input_shape = (img_rows, img_cols, 1)

Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

In [14]:
# 建造模型架構

model = Sequential()
model.add(Convolution2D(4, 5, 5,   # 4個filter,每個filter大小為5*5
          border_mode='valid',     # 不使用zero-padding
          subsample = (1,1),       # stride預設為1
          input_shape = input_shape))

model.add(Activation('relu'))      # activation function

model.add(Convolution2D(8, 3, 3,   # 8個filter,每個filter大小為3*3
          border_mode='same'))     # 使用zero-padding (須轉換backend為tensorflow,或安裝BLAS library)  

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))  # filter大小為2*2

model.add(Convolution2D(16, 3, 3, border_mode='valid')) 

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, init='normal'))  #fully-connected layer
model.add(Activation('relu'))
model.add(Dense(10, init='normal'))
model.add(Activation('softmax'))

In [15]:
# 定義什麼叫做學的好

sgd = SGD(lr=0.05,        # learning rate
          decay=1e-6,     # learning rate衰減的速度
          momentum=0.9)   # 慣性動能

model.compile(loss='categorical_crossentropy', # cost function
              optimizer='adam', # gradient decent method
              metrics=["accuracy"])

In [16]:
# 訓練 找出最佳解

model.fit(X_train, Y_train,
          batch_size=100, # 100張圖為一組
          nb_epoch=10,    # 訓練10輪
          shuffle=True,verbose=1,
          validation_split=0.2,  # 5折交叉驗證
          show_accuracy=True)

Train on 24000 samples, validate on 6000 samples
Epoch 1/10
24000/24000 [==============================] - 30s - loss: 0.4474 - acc: 0.8796 - val_loss: 0.1265 - val_acc: 0.9623
Epoch 2/10
24000/24000 [==============================] - 32s - loss: 0.1071 - acc: 0.9668 - val_loss: 0.1080 - val_acc: 0.9677
Epoch 3/10
24000/24000 [==============================] - 27s - loss: 0.0686 - acc: 0.9780 - val_loss: 0.0877 - val_acc: 0.9765
Epoch 4/10
24000/24000 [==============================] - 18s - loss: 0.0516 - acc: 0.9832 - val_loss: 0.1021 - val_acc: 0.9727
Epoch 5/10
24000/24000 [==============================] - 18s - loss: 0.0432 - acc: 0.9865 - val_loss: 0.1204 - val_acc: 0.9720
Epoch 6/10
24000/24000 [==============================] - 17s - loss: 0.0438 - acc: 0.9858 - val_loss: 0.1200 - val_acc: 0.9707
Epoch 7/10
24000/24000 [==============================] - 17s - loss: 0.0285 - acc: 0.9912 - val_loss: 0.1096 - val_acc: 0.9760
Epoch 8/10
24000/24000 [==============================]

In [17]:
# testing

score = model.evaluate(X_test, Y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

12000/12000 [==============================] - 8s     
Test score: 0.104586125199
Test accuracy: 0.980333333333


In [11]:
# save model weights

'''
WEIGHTS_FILEPATH = '/home/keras/Desktop/keras/mnist_cnn.hdf5'
MODEL_ARCH_FILEPATH = '/home/keras/Desktop/keras/mnist_cnn.json'

with open(MODEL_ARCH_FILEPATH, 'w') as f:
    f.write(model.to_json())
    
model.save_weights('/home/keras/Desktop/keras/mnist_cnn.hdf5')
'''